<a href="https://colab.research.google.com/github/nilu-ai/Restaurant_Fttx_project/blob/main/pre%20trained%20facenet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install deepface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 62.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.4/84.4 kB 9.5 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.6.0-py2.py3-none-any.whl size=117029 sha256=0241437da2ef6f979dc73c7215cd75f5203642a185ae57a56b368ac9a23e7f8a
  Stored in directory: /root/.cache/pip/wheels/d6/6d/5d/5b73fa0f46d01a793713f8859201361e9e581ced8c75e5c6a3
Successfully built fire


In [1]:
!unzip trainss

Archive:  trainss.zip
   creating: trainss/abhi/
  inflating: trainss/abhi/WIN_20240524_15_28_37_Pro.jpg  
  inflating: trainss/abhi/WIN_20240524_15_28_39_Pro.jpg  
  inflating: trainss/abhi/WIN_20240524_15_28_41_Pro.jpg  
  inflating: trainss/abhi/WIN_20240524_15_28_42_Pro.jpg  
  inflating: trainss/abhi/WIN_20240524_15_28_44_Pro.jpg  
  inflating: trainss/abhi/WIN_20240524_15_28_45_Pro.jpg  
  inflating: trainss/abhi/WIN_20240524_15_28_47_Pro.jpg  
  inflating: trainss/abhi/WIN_20240524_15_28_48_Pro.jpg  
  inflating: trainss/abhi/WIN_20240524_15_28_50_Pro.jpg  
  inflating: trainss/abhi/WIN_20240524_15_28_52_Pro.jpg  
   creating: trainss/Abhishek 1/
  inflating: trainss/Abhishek 1/WIN_20240524_15_28_24_Pro (2).jpg  
  inflating: trainss/Abhishek 1/WIN_20240524_15_28_24_Pro.jpg  
  inflating: trainss/Abhishek 1/WIN_20240524_15_28_25_Pro (2).jpg  
  inflating: trainss/Abhishek 1/WIN_20240524_15_28_25_Pro.jpg  
  inflating: trainss/Abhishek 1/WIN_20240524_15_29_22_Pro.jpg  
  inflatin

In [33]:
from deepface import DeepFace
predictions = DeepFace.find(img_path = "/content/trainss/Rohan/WIN_20240524_15_21_36_Pro.jpg", db_path = "/content/trainss",model_name="Facenet512")
# print(df)
# if df[0].distance.empty:
#   print("Unkonw")
# elif df[0].distance[0]<0.3:
#   print(df)
# else:
#   print("Unknow User")
import re

if not predictions[0].empty and predictions[0].distance[0] < 0.3:
  data=predictions[0].to_dict()
  print(data)
  response = {
                    'status': 'success',
                    'data': data["identity"][0].split("/")[3]

                }
else:
 response = {
                    'status': 'unknown User'
                }

print(response)


24-06-04 06:32:17 - Searching /content/trainss/Rohan/WIN_20240524_15_21_36_Pro.jpg in 36 length datastore
24-06-04 06:32:18 - find function duration 0.9366366863250732 seconds
{'identity': {0: '/content/trainss/Rohan/WIN_20240524_15_21_36_Pro.jpg'}, 'hash': {0: 'ee721f76c0af38081115b527cfb49d84f9f4ca32'}, 'target_x': {0: 536}, 'target_y': {0: 282}, 'target_w': {0: 270}, 'target_h': {0: 270}, 'source_x': {0: 536}, 'source_y': {0: 282}, 'source_w': {0: 270}, 'source_h': {0: 270}, 'threshold': {0: 0.3}, 'distance': {0: 0.0}}
{'status': 'success', 'data': 'Rohan'}


In [34]:
from flask import Flask, request, jsonify
import cv2
import numpy as np
import base64
from deepface import DeepFace
import os
app = Flask(__name__)
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(5000)"))
@app.route('/', methods=['GET', 'POST'])
def upload_image():
    if request.method == 'POST':
        data = request.json
        img_data = data['image'].split(',')[1]
        nparr = np.frombuffer(base64.b64decode(img_data), np.uint8)
        img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)

        try:

            predictions = DeepFace.find(img_path=img, db_path="/content/trainss", model_name="Facenet512")

            if not predictions[0].empty and predictions[0].distance[0] < 0.3:
                data=predictions[0].to_dict()
                response = {
                    'status': 'success',
                    #'data': predictions[0].to_dict(),
                    'prediction':data["identity"][0].split("/")[3]
                }
            else:
                response = {
                    'status': 'unknown User'
                }
        except Exception as e:
            response = {
                'status': 'error',
                'message': str(e)
            }

        return jsonify(response)

    return """
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
    </head>
    <body>
        <video id="video" width="640" height="480" autoplay></video>
        <canvas id="canvas" width="640" height="480" style="display:none;"></canvas>
        <div id="prediction"></div>
        <script>
            const video = document.getElementById('video');
            const canvas = document.getElementById('canvas');
            const predictionDiv = document.getElementById('prediction');

            async function initCamera() {
                try {
                    const stream = await navigator.mediaDevices.getUserMedia({ video: true });
                    video.srcObject = stream;
                } catch (err) {
                    console.error('Error accessing camera:', err);
                }
            }

            async function captureAndPredict() {
                const context = canvas.getContext('2d');
                context.drawImage(video, 0, 0, canvas.width, canvas.height);
                const imageData = canvas.toDataURL('image/jpeg');
                const response = await fetch('/', {
                    method: 'POST',
                    body: JSON.stringify({ image: imageData }),
                    headers: {
                        'Content-Type': 'application/json'
                    }
                });
                const result = await response.json();
                predictionDiv.innerHTML = JSON.stringify(result, null, 2);
            }

            setInterval(captureAndPredict, 1000);

            initCamera();
        </script>
    </body>
    </html>
    """


@app.route('/new_user', methods=['GET', 'POST'])
def newuser():
    if request.method == 'POST':
        data = request.get_json()
        username = data['username']
        photos = data['photos']

        # Create directory for the new user
        user_dir = os.path.join('trainss', username)
        os.makedirs(user_dir, exist_ok=True)

        # Save photos
        for i, photo in enumerate(photos):
            photo_data = base64.b64decode(photo.split(',')[1])
            with open(os.path.join(user_dir, f'photo_{i+1}.jpg'), 'wb') as f:
                f.write(photo_data)

        return jsonify({'status': 'success', 'message': 'User data saved.'})

    else:
        return '''
            <!DOCTYPE html>
            <html lang="en">
            <head>
                <meta charset="UTF-8">
                <meta name="viewport" content="width=device-width, initial-scale=1.0">
            </head>
            <body>
                <video id="video" width="640" height="480" autoplay></video>
                <canvas id="canvas" width="640" height="480" style="display:none;"></canvas>
                    <label>Name: <input id="username" type="text"/></label>
                <button id="submitimage">Submit New Data</button>
                <script>
                    const video = document.getElementById('video');
                    const canvas = document.getElementById('canvas');
                    const submit = document.getElementById('submitimage');
                    const usernameInput = document.getElementById('username');
                    let photos = [];
                    let photoCount = 0;

                    async function initCamera() {
                        try {
                            const stream = await navigator.mediaDevices.getUserMedia({ video: true });
                            video.srcObject = stream;
                        } catch (err) {
                            console.error('Error accessing camera:', err);
                        }
                    }

                    function capturePhoto() {
                        const context = canvas.getContext('2d');
                        context.drawImage(video, 0, 0, canvas.width, canvas.height);
                        const imageData = canvas.toDataURL('image/jpeg');
                        photos.push(imageData);
                    }

                    async function submitData() {
                        if (photoCount < 3) {
                            capturePhoto();
                            photoCount++;
                            setTimeout(submitData, 1000); // Capture a photo every second
                        } else {
                            const username = usernameInput.value;
                            const response = await fetch('/new_user', {
                                method: 'POST',
                                body: JSON.stringify({ username, photos }),
                                headers: {
                                    'Content-Type': 'application/json'
                                }
                            });
                            const result = await response.json();
                            console.log(result);
                        }
                    }

                    submit.addEventListener('click', () => {
                        photoCount = 0;
                        photos = [];
                        submitData();
                    });

                    initCamera();
                </script>
            </body>
            </html>
        '''



if __name__ == '__main__':
    app.run(host='0.0.0.0')

https://plmrfpv1u5-496ff2e9c6d22116-5000-colab.googleusercontent.com/
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [04/Jun/2024 06:32:59] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Jun/2024 06:33:00] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [04/Jun/2024 06:33:01] "POST / HTTP/1.1" 200 -


24-06-04 06:33:01 - Searching [[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]] in 36 length datastore


INFO:werkzeug:127.0.0.1 - - [04/Jun/2024 06:33:02] "POST / HTTP/1.1" 200 -


24-06-04 06:33:02 - Searching [[[130 137 140]
  [130 137 140]
  [131 138 141]
  ...
  [136 147 151]
  [133 145 147]
  [129 141 143]]

 [[130 137 140]
  [130 137 140]
  [130 137 140]
  ...
  [137 148 152]
  [134 146 148]
  [130 142 144]]

 [[130 137 140]
  [130 137 140]
  [130 137 140]
  ...
  [136 148 150]
  [131 143 145]
  [126 138 140]]

 ...

 [[193 173  95]
  [192 173 100]
  [195 176 115]
  ...
  [ 20  25  24]
  [ 23  28  27]
  [ 23  28  26]]

 [[192 173  92]
  [189 170  97]
  [194 174 116]
  ...
  [ 23  25  25]
  [ 22  27  26]
  [ 22  27  26]]

 [[193 174  93]
  [189 170  97]
  [194 174 116]
  ...
  [ 26  28  28]
  [ 23  28  27]
  [ 22  27  26]]] in 36 length datastore
24-06-04 06:33:03 - Searching [[[136 148 142]
  [136 148 142]
  [137 148 145]
  ...
  [141 156 158]
  [137 155 156]
  [135 153 154]]

 [[136 148 142]
  [137 149 143]
  [137 148 145]
  ...
  [139 154 156]
  [137 152 154]
  [133 151 152]]

 [[136 147 144]
  [136 147 144]
  [137 148 146]
  ...
  [141 154 156]
  [138 15

...

24-06-04 06:33:22 - Searching [[[181 187 192]
  [181 187 192]
  [181 187 192]
  ...
  [192 197 198]
  [193 197 198]
  [193 197 198]]

 [[181 187 192]
  [181 187 192]
  [181 187 192]
  ...
  [193 198 199]
  [194 198 199]
  [194 198 199]]

 [[181 187 192]
  [181 187 192]
  [181 187 192]
  ...
  [193 198 199]
  [195 199 200]
  [195 199 200]]

 ...

 [[255 255 239]
  [255 255 240]
  [255 253 235]
  ...
  [201 205 193]
  [199 205 188]
  [201 207 188]]

 [[255 255 243]
  [255 255 242]
  [255 252 237]
  ...
  [197 204 189]
  [192 201 181]
  [191 200 179]]

 [[255 253 242]
  [255 254 243]
  [255 253 239]
  ...
  [197 205 188]
  [183 192 172]
  [175 184 163]]] in 36 length datastore


INFO:werkzeug:127.0.0.1 - - [04/Jun/2024 06:33:23] "POST / HTTP/1.1" 200 -


24-06-04 06:33:23 - Searching [[[181 189 188]
  [181 189 188]
  [180 188 187]
  ...
  [194 197 201]
  [195 198 202]
  [195 198 202]]

 [[180 188 187]
  [180 188 187]
  [180 188 187]
  ...
  [194 197 201]
  [194 197 201]
  [195 198 202]]

 [[182 188 187]
  [182 188 187]
  [182 188 187]
  ...
  [194 197 201]
  [194 197 201]
  [194 197 201]]

 ...

 [[255 253 242]
  [255 255 241]
  [255 255 237]
  ...
  [207 204 190]
  [203 198 183]
  [204 200 182]]

 [[255 255 245]
  [255 254 243]
  [253 253 237]
  ...
  [204 203 189]
  [193 190 175]
  [188 185 170]]

 [[253 252 242]
  [254 254 242]
  [252 251 237]
  ...
  [204 203 189]
  [189 186 172]
  [181 178 163]]] in 36 length datastore


INFO:werkzeug:127.0.0.1 - - [04/Jun/2024 06:33:24] "POST / HTTP/1.1" 200 -


24-06-04 06:33:24 - Searching [[[186 190 191]
  [186 190 191]
  [185 189 190]
  ...
  [198 201 205]
  [197 200 204]
  [196 199 203]]

 [[186 190 191]
  [186 190 191]
  [186 190 191]
  ...
  [196 199 203]
  [196 199 203]
  [196 199 203]]

 [[187 191 192]
  [186 190 191]
  [186 190 191]
  ...
  [195 198 202]
  [195 198 202]
  [197 200 204]]

 ...

 [[255 255 238]
  [254 255 235]
  [255 255 239]
  ...
  [204 202 192]
  [199 197 186]
  [203 201 190]]

 [[255 253 242]
  [255 255 241]
  [255 254 243]
  ...
  [202 201 191]
  [190 189 179]
  [193 192 182]]

 [[255 255 247]
  [254 253 243]
  [246 245 235]
  ...
  [203 202 192]
  [183 182 172]
  [185 184 174]]] in 36 length datastore
